In [52]:
import numpy as np
import pandas as pd

In [53]:
df = pd.read_csv('/content/drive/MyDrive/SENG 607/Typo3/processedDataTypo3.csv')
df.head()

,Unnamed: 0,Project,created_on,dependencies,desc,id,relationCount,status,strtDate,subject,tracker
0,0,TYPO3 Core,2021-01-14T08:23:43Z,89301relates93288#93287relates93288#,"If URLs are changed several times, redirect ch...",93288,2,New,2021-01-14,Merge redirect chains,Feature
1,1,TYPO3 Core,2021-01-11T15:04:55Z,NaN,"Since version 10.3, webp can also be used as a...",93264,0,New,2021-01-11,Add webp check,Feature
2,2,TYPO3 Core,2021-01-08T15:22:50Z,NaN,"Options for ""Textmedia"" set by ""Media Adjustme...",93247,0,New,2021-01-08,"Modernize ""Textmedia"" and extend ""Media Adjust...",Feature
3,3,TYPO3 Core,2021-01-04T10:05:37Z,NaN,"Hi,\r\n\r\nIt would be great to add a method i...",93210,0,New,2021-01-04,Allow refreshing single widget,Feature
4,4,TYPO3 Core,2021-01-03T22:59:44Z,NaN,Working with Files and Folders in FAL is quite...,93209,0,Under Review,2021-01-03,Add getFile() to TYPO3\CMS\Core\Resource\Folder,Feature


In [54]:
df.drop(['Unnamed: 0','Project','created_on','status','strtDate','subject','tracker'],inplace=True,axis=1)

In [55]:
print(df.shape)
print(df.isnull().sum())

(9291, 4)
dependencies     6678
desc              686
id                  0
relationCount       0
dtype: int64


In [56]:
df.dropna(subset=['dependencies','desc'],inplace=True)
df = df.reset_index(drop=True)

In [57]:
df.head()

,dependencies,desc,id,relationCount
0,89301relates93288#93287relates93288#,"If URLs are changed several times, redirect ch...",93288,2
1,87457relates93197#,Right now only the property @var annotation ca...,93197,1
2,75043relates93182#,Make the compressed file extension configurabl...,93182,1
3,93170relates93171#,Introduce a fallback site configuration to be ...,93171,1
4,93170relates93171#,I would suggest to deprecate the following pag...,93170,1


In [58]:
print(df.shape)
print(df.isnull().sum())

(2477, 4)
dependencies     0
desc             0
id               0
relationCount    0
dtype: int64


**Creating Dependent Pairs**

In [59]:
import re
id1 = []
id2 = []
desc1 = []
desc2 = []
dependency  = []

for i in range(2477):
  a = df['dependencies'][i]
  a = a.split('#')
  for j in range(len(a)-1):
    b = a[j]
    temp = re.findall(r'\d+', b)                         # For extracting id's
    r = list(map(int, temp))
    id1.append(r[0])
    id2.append(r[1])

    c = df[df['id']==r[0]]['desc'].to_string()          # For extracting description wrt to each id
    c1 = df[df['id']==r[1]]['desc'].to_string() 
    desc1.append(c[5:])
    desc2.append(c1[5:])

    temp1 = str(r[0])
    temp2 = str(r[1])
    result = re.search(f'{temp1}(.*){temp2}',b).group(1) # For extracting type of dependency
    dependency.append(result)

In [60]:
df1 = pd.DataFrame({'id1':id1,'Description1':desc1,'id2':id2,'Description2':desc2,'Type of dependency':dependency})
df1.head(10)

,id1,Description1,id2,Description2,Type of dependency
0,89301,"s([], )",93288,"If URLs are changed several times, redirect ch...",relates
1,93287,"s([], )",93288,"If URLs are changed several times, redirect ch...",relates
2,87457,This patch introduces the requirement to sym...,93197,Right now only the property @var annotation ca...,relates
3,75043,Please use .gz instead of .gzip for statical...,93182,Make the compressed file extension configurabl...,relates
4,93170,I would suggest to deprecate the following pag...,93171,Introduce a fallback site configuration to be ...,relates
5,93170,I would suggest to deprecate the following pag...,93171,Introduce a fallback site configuration to be ...,relates
6,92553,"s([], )",93162,The sysext *redirects* allows for redirects to...,relates
7,75266,There should be a possibility to have at lea...,92903,It would be great to be able to arrange FlexFo...,relates
8,89509,Fluid does not provide a way to access flexfo...,92875,The tCA allows the field flex. I want use some...,relates
9,92297,This is one of several possible solution to i...,92822,"As already mentioned the problem of ""false pos...",relates


In [61]:
df1.shape

(4092, 5)

In [62]:
df1['Type of dependency'].value_counts()

relates       3232
duplicates     636
precedes       151
blocks          70
copied_to        3
Name: Type of dependency, dtype: int64

In [63]:
# Deleting empty lists
l1 = []
l2 = []
for i in range(4092):
  if df1['Description1'][i] == 's([], )':
    l1.append(i)
  if df1['Description2'][i] == 's([], )':
    l2.append(i)
l = list(set(l1+l2))
print(len(l))

df1.drop(l,inplace = True)
df1 = df1.reset_index(drop = True)

2270


In [64]:
df1.head(10)

,id1,Description1,id2,Description2,Type of dependency
0,87457,This patch introduces the requirement to sym...,93197,Right now only the property @var annotation ca...,relates
1,75043,Please use .gz instead of .gzip for statical...,93182,Make the compressed file extension configurabl...,relates
2,93170,I would suggest to deprecate the following pag...,93171,Introduce a fallback site configuration to be ...,relates
3,93170,I would suggest to deprecate the following pag...,93171,Introduce a fallback site configuration to be ...,relates
4,75266,There should be a possibility to have at lea...,92903,It would be great to be able to arrange FlexFo...,relates
5,89509,Fluid does not provide a way to access flexfo...,92875,The tCA allows the field flex. I want use some...,relates
6,92297,This is one of several possible solution to i...,92822,"As already mentioned the problem of ""false pos...",relates
7,36075,In the original concept for the new Page-Tr...,92576,h2. Use case: \r\n\r\nIt is possible to filte...,relates
8,89177,The linkvalidator's default PageTsConfig cur...,92542,We repeatedly have errors in the log due to i...,relates
9,92432,<pre>\r\nImported from T3cm 2020 (https://not...,66343,"At the T3UXW15 we get the feedback, that hot...",duplicates


In [65]:
print(df1.shape)
print(df1.isnull().sum())

(1822, 5)
id1                   0
Description1          0
id2                   0
Description2          0
Type of dependency    0
dtype: int64


In [66]:
# Sentence length analysis
y1,y2 = [],[]
y = df1['Description1']
z = df1['Description2']
for i in range(len(y)):
  q = y[i].split(' ')
  if len(q) <2:
    y1.append(i)
for i in range(len(y)):
  q1 = z[i].split(' ')
  if len(q1) <2:
    y2.append(i)
print(len(y1))
print(len(y2))
q2 = list(set(y1+y2))
df1.drop(q2,inplace=True)
df1 = df1.reset_index(drop=True)

0
0


**Final Depenent Pairs**

In [67]:
df1.head()

,id1,Description1,id2,Description2,Type of dependency
0,87457,This patch introduces the requirement to sym...,93197,Right now only the property @var annotation ca...,relates
1,75043,Please use .gz instead of .gzip for statical...,93182,Make the compressed file extension configurabl...,relates
2,93170,I would suggest to deprecate the following pag...,93171,Introduce a fallback site configuration to be ...,relates
3,93170,I would suggest to deprecate the following pag...,93171,Introduce a fallback site configuration to be ...,relates
4,75266,There should be a possibility to have at lea...,92903,It would be great to be able to arrange FlexFo...,relates


In [69]:
print(df1.shape)
print(df1.isnull().sum())
print(df1['Type of dependency'].value_counts())

(1822, 5)
id1                   0
Description1          0
id2                   0
Description2          0
Type of dependency    0
dtype: int64
relates       1372
duplicates     372
precedes        42
blocks          34
copied_to        2
Name: Type of dependency, dtype: int64


In [70]:
#df1.to_csv('/content/drive/MyDrive/SENG 607/Typo3/Typo3_Pos_modified.csv')

**Creating independent pairs**

In [71]:
df2 = pd.read_csv('/content/drive/MyDrive/SENG 607/Typo3/processedDataTypo3.csv')
df2.drop(['Unnamed: 0','Project','created_on','status','strtDate','subject','tracker'],inplace=True,axis=1)
df2.dropna(subset=['desc'],inplace=True)
df2.head()

,dependencies,desc,id,relationCount
0,89301relates93288#93287relates93288#,"If URLs are changed several times, redirect ch...",93288,2
1,NaN,"Since version 10.3, webp can also be used as a...",93264,0
2,NaN,"Options for ""Textmedia"" set by ""Media Adjustme...",93247,0
3,NaN,"Hi,\r\n\r\nIt would be great to add a method i...",93210,0
4,NaN,Working with Files and Folders in FAL is quite...,93209,0


In [72]:
df2['relationCount'].value_counts()

0     6128
1     1566
2      568
3      191
4       74
5       37
6       17
8        7
7        5
9        5
15       2
12       2
18       1
10       1
16       1
Name: relationCount, dtype: int64

In [73]:
i1 = []
i2 = []
d1 = []
d2 = []

n = df2[df2['relationCount']==0]['id'].values

for i in range(10000):
  d = np.random.choice(n,size=2)
  i1.append(d[0])
  i2.append(d[1])

  c = df2[df2['id']==d[0]]['desc'].to_string()          # For extracting description wrt to each id
  c1 = df2[df2['id']==d[1]]['desc'].to_string()
  d1.append(c[5:])
  d2.append(c1[5:])

In [74]:
df3 = pd.DataFrame({'id1':i1,'Description1':d1,'id2':i2,'Description2':d2})
df3.head()

,id1,Description1,id2,Description2
0,40046,Current BE user could have an option to sen...,21660,Currently the password change facility does...
1,27400,At the moment [-] and [+] are hardcoded in ...,16778,"Imo, there should be the possibility to ass..."
2,2589,It would be really great if the spellchecke...,85994,It is currently possible to create or modify...
3,54954,This extension has two problems with upcomm...,23259,This feature request unifies the two menu i...
4,61366,Hi! \r\n\r\nIt would be great if you could ...,15515,coudl it possible a simple hilight on rolle...


In [75]:
def ext(x):
  return 'independent'
df3['Type of dependency'] = df3['id1'].apply(ext)
df3.head()

,id1,Description1,id2,Description2,Type of dependency
0,40046,Current BE user could have an option to sen...,21660,Currently the password change facility does...,independent
1,27400,At the moment [-] and [+] are hardcoded in ...,16778,"Imo, there should be the possibility to ass...",independent
2,2589,It would be really great if the spellchecke...,85994,It is currently possible to create or modify...,independent
3,54954,This extension has two problems with upcomm...,23259,This feature request unifies the two menu i...,independent
4,61366,Hi! \r\n\r\nIt would be great if you could ...,15515,coudl it possible a simple hilight on rolle...,independent


In [76]:
# Sentence length analysis
y1,y2 = [],[]
y = df3['Description1']
z = df3['Description2']
for i in range(len(y)):
  q = y[i].split(' ')
  if len(q) <2:
    y1.append(i)
for i in range(len(y)):
  q1 = z[i].split(' ')
  if len(q1) <2:
    y2.append(i)
print(len(y1))
print(len(y2))
q2 = list(set(y1+y2))
df3.drop(q2,inplace=True)
df3 = df3.reset_index(drop=True)

0
0


**Final independent pairs**

In [77]:
df3.head()

,id1,Description1,id2,Description2,Type of dependency
0,40046,Current BE user could have an option to sen...,21660,Currently the password change facility does...,independent
1,27400,At the moment [-] and [+] are hardcoded in ...,16778,"Imo, there should be the possibility to ass...",independent
2,2589,It would be really great if the spellchecke...,85994,It is currently possible to create or modify...,independent
3,54954,This extension has two problems with upcomm...,23259,This feature request unifies the two menu i...,independent
4,61366,Hi! \r\n\r\nIt would be great if you could ...,15515,coudl it possible a simple hilight on rolle...,independent


In [78]:
print(df3.shape)
print(df3.isnull().sum())

(10000, 5)
id1                   0
Description1          0
id2                   0
Description2          0
Type of dependency    0
dtype: int64


In [79]:
#df3.to_csv('/content/drive/MyDrive/SENG 607/Typo3/Typo3_Neg_modified.csv')

**Processed Dependent Pairs**

In [ ]:
df4 = pd.read_csv('/content/drive/MyDrive/SENG 607/Typo3/Typo3_Processed_Pos_init.csv')
df4.drop(['Unnamed: 0'],axis=1,inplace=True)
df4.head()

,id1,Description1,id2,Description2,Type of dependency
0,87457,patch introduces requirement symfo,93197,right property var annotation ca,relates
1,75043,please use gz instead gzip statically,93182,make compressed file extension configurabl,relates
2,93170,would suggest deprecate following pag,93171,introduce fallback site configuration,relates
3,93170,would suggest deprecate following pag,93171,introduce fallback site configuration,relates
4,75266,possibility least,92903,would great able arrange flexfo,relates


In [ ]:
df4.isnull().sum()

id1                   0
Description1          6
id2                   0
Description2          2
Type of dependency    0
dtype: int64

In [ ]:
df4.dropna(inplace=True)
df4 = df4.reset_index(drop=True)

In [ ]:
y1,y2 = [],[]
y = df4['Description1']
z = df4['Description2']
for i in range(len(y)):
  q = y[i].split(' ')
  if len(q) <2:
    y1.append(i)
for i in range(len(y)):
  q1 = z[i].split(' ')
  if len(q1) <2:
    y2.append(i)
print(len(y1))
print(len(y2))
q2 = list(set(y1+y2))
df4.drop(q2,inplace=True)
df4 = df4.reset_index(drop=True)

28
26


In [ ]:
def isEnglish(s):
  return s.isascii()

a1 , a2 = [] , []
a = df4['Description1'].values
b = df4['Description2'].values

for i in range(len(a)):
  if isEnglish(a[i])==False:
    a1.append(i)
for j in range(len(b)):
  if isEnglish(b[j])==False:
    a2.append(j)

print(len(a1))
print(len(a2))
a3 = list(set(a1+a2))
print(len(a3))

df4.drop(a3,inplace=True)
df4 = df4.reset_index(drop=True)

2
6
8


**Final Processed Dependent**

In [ ]:
df4.head()

,id1,Description1,id2,Description2,Type of dependency
0,87457,patch introduces requirement symfo,93197,right property var annotation ca,relates
1,75043,please use gz instead gzip statically,93182,make compressed file extension configurabl,relates
2,93170,would suggest deprecate following pag,93171,introduce fallback site configuration,relates
3,93170,would suggest deprecate following pag,93171,introduce fallback site configuration,relates
4,75266,possibility least,92903,would great able arrange flexfo,relates


In [ ]:
print(df4.shape)
print(df4.isnull().sum())

(1752, 5)
id1                   0
Description1          0
id2                   0
Description2          0
Type of dependency    0
dtype: int64


In [ ]:
df4['Type of dependency'].value_counts()

relates       1324
duplicates     356
precedes        38
blocks          32
copied_to        2
Name: Type of dependency, dtype: int64

In [ ]:
#df4.to_csv('/content/drive/MyDrive/SENG 607/Typo3/Typo3_Processed_ALPos.csv')

**Processed independent pairs**

In [ ]:
df5 = pd.read_csv('/content/drive/MyDrive/SENG 607/Typo3/Typo3_Processed_Neg_init.csv')
df5.drop(['Unnamed: 0'],axis=1,inplace=True)
df5.head()

,id1,Description1,id2,Description2,Type of dependency
0,91745,comernthxrnstefan,6354,implementfinish customer address implem,independent
1,80689,result poll,71251,flash message different queue dependin,independent
2,13515,workspace empty notification dropdown sh,3109,seems supported htmlmail,independent
3,3264,hirnrnim sure standalone ver,50136,svg missing image icon svg iconpngr,independent
4,13948,need banner announces,12348,would great implement pagebrowser,independent


In [ ]:
df5.isnull().sum()

id1                    0
Description1          95
id2                    0
Description2          86
Type of dependency     0
dtype: int64

In [ ]:
df5.dropna(inplace=True)
df5 = df5.reset_index(drop=True)

In [ ]:
y1,y2 = [],[]
y = df5['Description1']
z = df5['Description2']
for i in range(len(y)):
  q = y[i].split(' ')
  if len(q) <2:
    y1.append(i)
for i in range(len(y)):
  q1 = z[i].split(' ')
  if len(q1) <2:
    y2.append(i)
print(len(y1))
print(len(y2))
q2 = list(set(y1+y2))
df5.drop(q2,inplace=True)
df5 = df5.reset_index(drop=True)

199
153


In [ ]:
u,u1 = [],[]
drop = [27908,38097,53148,33591]
for i in drop:
  a = df5[df5['id2']==i]['Description2'].index
  a = list(a)
  for j in a:
    u.append(j)
print(len(u))

for i in drop:
  b = df5[df5['id1']==i]['Description1'].index
  b = list(b)
  for j in b:
    u1.append(j)
print(len(u1))
s = list(set(u1+u))
print(len(s))

df5.drop(s,inplace=True)
df5 = df5.reset_index(drop=True)

12
4
16


In [ ]:
def isEnglish(s):
  return s.isascii()

a1 , a2 = [] , []
a = df5['Description1'].values
b = df5['Description2'].values

for i in range(len(a)):
  if isEnglish(a[i])==False:
    a1.append(i)
for j in range(len(b)):
  if isEnglish(b[j])==False:
    a2.append(j)

print(len(a1))
print(len(a2))
a3 = list(set(a1+a2))
print(len(a3))

df5.drop(a3,inplace=True)
df5 = df5.reset_index(drop=True)

72
80
152


**Final Processed independent**

In [ ]:
df5.head()

,id1,Description1,id2,Description2,Type of dependency
0,80689,result poll,71251,flash message different queue dependin,independent
1,13515,workspace empty notification dropdown sh,3109,seems supported htmlmail,independent
2,3264,hirnrnim sure standalone ver,50136,svg missing image icon svg iconpngr,independent
3,13948,need banner announces,12348,would great implement pagebrowser,independent
4,77336,currently way inject li,1011,optionally show select link,independent


In [ ]:
print(df5.shape)
print(df5.isnull().sum())

(9270, 5)
id1                   0
Description1          0
id2                   0
Description2          0
Type of dependency    0
dtype: int64


In [ ]:
#df5.to_csv('/content/drive/MyDrive/SENG 607/Typo3/Typo3_Processed_ALNeg.csv')

**BERT**

In [87]:
df6 = pd.read_csv('/content/drive/MyDrive/SENG 607/Typo3/Typo3_Processed_BERTPos_init_modified.csv')
df6.drop(['Unnamed: 0'],axis=1,inplace=True)
df6.head()

,id1,Description1,id2,Description2,Type of dependency
0,87457,this patch introduces the requirement to symfo,93197,right now only the property var annotation ca,relates
1,75043,please use gz instead of gzip for statically,93182,make the compressed file extension configurabl,relates
2,93170,i would suggest to deprecate the following pag,93171,introduce fallback site configuration to be,relates
3,93170,i would suggest to deprecate the following pag,93171,introduce fallback site configuration to be,relates
4,75266,there should be possibility to have at least,92903,it would be great to be able to arrange flexfo,relates


In [88]:
df6.isnull().sum()

id1                   0
Description1          8
id2                   0
Description2          2
Type of dependency    0
dtype: int64

In [89]:
df6.dropna(inplace=True)
df6 = df6.reset_index(drop=True)

In [90]:
def isEnglish(s):
  return s.isascii()

a1 , a2 = [] , []
a = df6['Description1'].values
b = df6['Description2'].values

for i in range(len(a)):
  if isEnglish(a[i])==False:
    a1.append(i)
for j in range(len(b)):
  if isEnglish(b[j])==False:
    a2.append(j)

print(len(a1))
print(len(a2))
a3 = list(set(a1+a2))
print(len(a3))

df6.drop(a3,inplace=True)
df6 = df6.reset_index(drop=True)

2
6
8


In [91]:
df6.isnull().sum()

id1                   0
Description1          0
id2                   0
Description2          0
Type of dependency    0
dtype: int64

In [92]:
# Sentence Length Analysis
y1,y2 = [],[]
y = df6['Description1']
z = df6['Description2']
for i in range(len(y)):
  q = y[i].split(' ')
  if len(q) <2:
    y1.append(i)
for i in range(len(y)):
  q1 = z[i].split(' ')
  if len(q1) <2:
    y2.append(i)
print(len(y1))
print(len(y2))
q2 = list(set(y1+y2))
df6.drop(q2,inplace=True)
df6 = df6.reset_index(drop=True)

8
4


In [93]:
df6.head()

,id1,Description1,id2,Description2,Type of dependency
0,87457,this patch introduces the requirement to symfo,93197,right now only the property var annotation ca,relates
1,75043,please use gz instead of gzip for statically,93182,make the compressed file extension configurabl,relates
2,93170,i would suggest to deprecate the following pag,93171,introduce fallback site configuration to be,relates
3,93170,i would suggest to deprecate the following pag,93171,introduce fallback site configuration to be,relates
4,75266,there should be possibility to have at least,92903,it would be great to be able to arrange flexfo,relates


In [94]:
#df6.to_csv('/content/drive/MyDrive/SENG 607/Typo3/Typo3_Processed_BERTPos_modified.csv')

In [102]:
df7 = pd.read_csv('/content/drive/MyDrive/SENG 607/Typo3/Typo3_Processed_BERTNeg_init_modified.csv')
df7.drop(['Unnamed: 0'],axis=1,inplace=True)
df7.head()

,id1,Description1,id2,Description2,Type of dependency
0,40046,current be user could have an option to send n,21660,currently the password change facility does no,independent
1,27400,at the moment and are hardcoded in,16778,imo there should be the possibility to assign,independent
2,2589,it would be really great if the spellchecker p,85994,it is currently possible to create or modify s,independent
3,54954,this extension has two problems with upcomming,23259,this feature request unifies the two menu item,independent
4,61366,hi rnrnit would be great if you could add,15515,coudl it possible simple hilight on rollever,independent


In [103]:
df7.isnull().sum()

id1                     0
Description1           94
id2                     0
Description2          105
Type of dependency      0
dtype: int64

In [104]:
df7.dropna(inplace=True)
df7 = df7.reset_index(drop=True)

In [105]:
def isEnglish(s):
  return s.isascii()

a1 , a2 = [] , []
a = df7['Description1'].values
b = df7['Description2'].values

for i in range(len(a)):
  if isEnglish(a[i])==False:
    a1.append(i)
for j in range(len(b)):
  if isEnglish(b[j])==False:
    a2.append(j)

print(len(a1))
print(len(a2))
a3 = list(set(a1+a2))
print(len(a3))

df7.drop(a3,inplace=True)
df7 = df7.reset_index(drop=True)

90
71
160


In [106]:
df7.isnull().sum()

id1                   0
Description1          0
id2                   0
Description2          0
Type of dependency    0
dtype: int64

In [107]:
# Sentence Length Analysis
y1,y2 = [],[]
y = df7['Description1']
z = df7['Description2']
for i in range(len(y)):
  q = y[i].split(' ')
  if len(q) <2:
    y1.append(i)
for i in range(len(y)):
  q1 = z[i].split(' ')
  if len(q1) <2:
    y2.append(i)
print(len(y1))
print(len(y2))
q2 = list(set(y1+y2))
df7.drop(q2,inplace=True)
df7 = df7.reset_index(drop=True)

104
104


In [108]:
df7.head()

,id1,Description1,id2,Description2,Type of dependency
0,40046,current be user could have an option to send n,21660,currently the password change facility does no,independent
1,27400,at the moment and are hardcoded in,16778,imo there should be the possibility to assign,independent
2,2589,it would be really great if the spellchecker p,85994,it is currently possible to create or modify s,independent
3,54954,this extension has two problems with upcomming,23259,this feature request unifies the two menu item,independent
4,61366,hi rnrnit would be great if you could add,15515,coudl it possible simple hilight on rollever,independent


In [109]:
#df7.to_csv('/content/drive/MyDrive/SENG 607/Typo3/Typo3_Processed_BERTNeg_modified.csv')

In [110]:
df7.isnull().sum()

id1                   0
Description1          0
id2                   0
Description2          0
Type of dependency    0
dtype: int64